**Algoritmo Evolution Strategies em Python**

**Autor**: Iran Freitas Ribeiro

**Disciplina**: Computação Natural

**Professor**: Renato A. Krohling

Implementação baseada no artigo: *A clustering method combining differential evolution with the K-means algorithm*

In [ ]:
import numpy as np
import pandas as pd
import functions as f
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Funções

In [27]:
def inicial_cluster(data,K):
    """
    Calcula o cluster inicial

    Params:
        - data: dados de treino
        - K: numero de clusters
    
    return:
        - centroids: centroids obtidos

    """

    ## escolhe um vetor aleatorio de X como o primeiro cluster
    # centroids escolhidos
    indices = list(np.arange(0,len(data)))
    cent_ = []
    id_m1 = np.random.choice(indices,size=1)[0]
    cent_.append(id_m1)
        
    # centroid inicial
    centroids = [data[id_m1]]
    # vetor de distâncias
    d_x = []
    for k in range(1,K):
        ## seleciona t=20 vetores de X ainda não escolhidos
        
        # atualiza lista de indices
        indices = list(set(indices).difference(cent_))
        id_t = np.random.choice(indices, size=30, replace=False)
        idc = []
        for j in id_t:
            cen_data = data[j]
            for c_ in centroids:
                if not (c_ == cen_data).all():
                    idc.append(j)
            # sai do loop se a quantidade minima foi obtida
            if len(idc)==20:
                break
        # pega os centroids correspondentes
        X = data[idc]
        d_x=[]
        for i, x in enumerate(X):
            distancias = []
            for c in centroids:
                d = np.linalg.norm(x - c).sum()
                distancias.append(d)
            d_x.append (np.min(distancias))
        id_k = np.argmax(d_x)
        centroids.append(X[id_k])
    return np.array(centroids)

def inicia_populacao(N, bounds, n_clusters, data):
    """
    Inicia a população utilizando executando o kmeans uma vez
    Params:
        -N: tamanho da população
        -n_clusters: número de clusters
        -data: dados de treino
    return:
        - u: população de indivíduos
    """
    
    cls_inicial = inicial_cluster(data, n_clusters)
    kmeans = KMeans(n_clusters, init=cls_inicial, max_iter=1, n_init=1)
    u = [kmeans.fit(data).cluster_centers_ for _ in range(N)]
    return u

def otimizacao_local(U_t,n_clusters, data):
    """
    Executa o kmeans iniciado com um centroid da população

    Params:
        - U_t: individuo da população
        - n_clusters: número de clusters
        - data: dados de treino
    return:
        - novo_ut: retorna o individuo otimizado
    """
    kmeans = KMeans(n_clusters=n_clusters, init=U_t, n_init=1, max_iter=1)
    kmeans = kmeans.fit(data)
    novo_Ut = kmeans.cluster_centers_
    return novo_Ut    

def DE(X, N=100, n_gen = 100, n_clusters=3, bounds=[0,1], F=0.01, CR=0.01, lograte = 10):
    """
    Executa o algoritmo de Differential Evolution
    
    Params:
        - X: base de dados
        - N: tamanho da população de indivíduos
        - n_gen: numero de gerações
        - n_clusters: número de clusters
        - bounds: limites do espaço de busca
        - CR: taxa de mutação
        - lograte: taxa de exibição dos melhores resultados
    return:
        - U[best]: melhor individuo da população
        - best_score: melhor score encontrado
        - historico_geracoes: historico de scores
    """
    # inicia a população
    U = inicia_populacao(N, n_clusters,X)
    # inicialização do melhor individuo e melhor score
    best = 0
    best_score = np.inf
    historico_geracoes = []
    # inicia parâmetros da população original
    Fu = [0.05 + np.random.uniform()*0.35 for i in range(N)]
    for g in range(n_gen):
        # geração da população temporária
        U_t, Fy = f.reproducao_crossover(U,Fu,CR)
        # compara cada individuo da população original com a temporária
        for i in range(len(U_t)):
            
            # otimização local do individuo temporario                
            novoU_t = otimizacao_local(U_t[i],n_clusters,X)
            
            # score do individuo temporario
            pred_cluster_ut = f.predict(X, novoU_t)
            score_ut = f.eval_individual(novoU_t,pred_cluster_ut,X)
            # score do individuo original
            pred_cluster = f.predict(X, U[i])
            score_u = f.eval_individual(U[i], pred_cluster, X)
            # verifica melhor score
            if (score_ut<score_u):
                U[i] = novoU_t
                Fu[i] = Fy[i]
                if (score_ut<best_score):
                    best_score = score_ut
                    best = i
            if (score_u<best_score):
                best_score = score_u
                best = i
        # salva melhor score na geração atual
        historico_geracoes.append(best_score)
        if ((lograte>0) and (g%lograte==0)):
            print ("best: {} SSE: {}".format(best,best_score))
    return U[best], best_score, historico_geracoes

In [ ]:
def experimentos(n_testes, exp_name = "", args=[]):
    """
    Realiza os experimentos

    Params:
        - n_testes: número de execuções
        - exp_name: nome do experimento
        - args: lista de argumentos
    
    return:
        - best_exp: melhor individuo 
        - best_score_exp: score do melhor individuo
        - best_historico: historico de scores do melhor individuo

    """
    best_exp = None
    best_score_exp = np.inf
    best_historico = []
    list_bests = []
    list_scores = []
    for i in range(n_testes):
        best, best_score, historico = DE(args[0],args[1],args[2],args[3],args[4],args[5],args[6],lograte=-1)
        list_bests.append(best)
        list_scores.append(best_score)
        best_historico.append(historico)
        if (best_score<best_score_exp):
            best_score_exp = best_score
            best_exp = best
        print ("{} - best_score {}".format(i, best_score_exp))
        
    np.save('results/DE_Kmeans/best_historico_{}.npy'.format(exp_name), np.array(best_historico))
    np.save('results/DE_Kmeans/list_scores_{}.npy'.format(exp_name), np.array(list_scores))
    np.save('results/DE_Kmeans/best_exp_{}.npy'.format(exp_name),best_exp)
    return best_exp, best_score_exp, best_historico

# IRIS

In [ ]:
irisdata,targets = f.load_iris()
X_iris = irisdata[[0,1,2,3]].values

In [ ]:
bounds = [0.1,7.9]
n_clusters = 3
N = 100
n_gen = 100
F = 0.2
CR = 0.2
args=[X_iris,N,n_gen,n_clusters,bounds,F,CR]
#_, _, _ = DE(args[0],args[1],args[2],args[3],args[4],args[5],args[6],lograte=10)

In [ ]:
best, best_score, historicoDE = experimentos(10, exp_name="DE_kmeans_iris",args=[X_iris,N,n_gen,n_clusters,bounds,F,CR])

best_score 97.34621969415682
best_score 97.32592423430009
best_score 97.32592423430009
best_score 97.32592423430009
best_score 97.32592423430009
best_score 97.32592423430009
best_score 97.32592423430009
best_score 97.32592423430009
best_score 97.32592423430009
best_score 97.32592423430009


# WINE

In [ ]:
wine_data, wine_targets = f.load_wine()
wine_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [ ]:
scaler = MinMaxScaler()
wine_data_test= scaler.fit_transform(wine_data[np.arange(1,14)])

In [ ]:
X_wine = wine_data_test
bounds = [np.min(X_wine),np.max(X_wine)]
n_clusters = 3
N = 100
n_gen = 100
F = 0.2
CR = 0.2

In [ ]:
args=[X_wine,N,100,n_clusters,bounds,0.2,0.03]
best_wine, best_score_wine, _ = DE(args[0],args[1],args[2],args[3], args[4],args[5],args[6],lograte=25)

In [21]:
bounds = [np.min(X_wine),np.max(X_wine)]
n_clusters = len(wine_targets)
N = 200
n_gen = 100
F = 0.2
CR = 0.2
args_wine=[X_wine,N,200,n_clusters,bounds,0.2,0.1]
best, best_score, historicoDE = experimentos(10, exp_name="DE_Kmeans_wine",args=args_wine)

0 - best_score 88.71884397824732
1 - best_score 88.71884397824732
2 - best_score 88.69568510213014
3 - best_score 88.69568510213014
4 - best_score 88.69568510213014
5 - best_score 88.69568510213014
6 - best_score 88.69568510213014
7 - best_score 88.69568510213014
8 - best_score 88.69568510213014
9 - best_score 88.69568510213014


# Breast Cancer

In [22]:
breast_cancer, targest_breast = f.load_breast_cancer()
colunas = breast_cancer.columns
# labels de cada linha da base de dados
labels = breast_cancer['diagnosis'].values
# seleciona apenas as features reais e pega os valores
data_breast_cancer = breast_cancer[colunas[1:]]
X_data_breast = data_breast_cancer.values

In [23]:
data_breast_cancer.describe()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,0.405172,1.216853,2.866059,40.337079,0.007041,0.025478,0.031894,0.011796,0.020542,0.003795,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,0.277313,0.551648,2.021855,45.491006,0.003003,0.017908,0.030186,0.006170,0.008266,0.002646,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,0.111500,0.360200,0.757000,6.802000,0.001713,0.002252,0.000000,0.000000,0.007882,0.000895,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,0.232400,0.833900,1.606000,17.850000,0.005169,0.013080,0.015090,0.007638,0.015160,0.002248,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,0.324200,1.108000,2.287000,24.530000,0.006380,0.020450,0.025890,0.010930,0.018730,0.003187,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,0.478900,1.474000,3.357000,45.190000,0.008146,0.032450,0.042050,0.014710,0.023480,0.004558,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,2.873000,4.885000,21.980000,542.200000,0.031130,0.135400,0.396000,0.052790,0.078950,0.029840,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [24]:
# transforma os dados para o intervalo [0,1]
scaler_breast = MinMaxScaler()
X_data_breast = scaler_breast.fit_transform(X_data_breast)

In [28]:
bounds = [np.min(X_data_breast),np.max(X_data_breast)]
n_clusters = len(targest_breast)
N = 500
n_gen = 200
F = 0.2
CR = 0.1
args_breast=[X_data_breast,N,n_gen,n_clusters,bounds,0.2,CR]
best, best_score, historicoDE = experimentos(10, exp_name="DE_Kmeans_breast",args=args_breast)

0 - best_score 318.30588325673534
1 - best_score 318.30588325673534
2 - best_score 318.30588325673534
3 - best_score 318.30588325673534
4 - best_score 318.30588325673534
5 - best_score 318.00130476242646
6 - best_score 318.00130476242646
7 - best_score 317.89009308172274
8 - best_score 317.80233378115304
9 - best_score 317.80233378115304
